In [32]:
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [33]:
# CSV 파일에서 URL 불러오기
df = pd.read_csv('./article_details.csv')

In [34]:
df.columns

Index(['title', 'summary', 'date', 'body', 'url', 'content'], dtype='object')

In [38]:
import jpype

jvmpath = "/Library/Java/JavaVirtualMachines/adoptopenjdk-14.jdk/Contents/Home/lib/libjli.dylib"

if not jpype.isJVMStarted():
    jpype.startJVM(jvmpath, '-Dfile.encoding=UTF8')

okt = Okt()

FileNotFoundError: [Errno 2] JVM DLL not found: /Library/Java/JavaVirtualMachines/adoptopenjdk-14.jdk/Contents/Home/lib/libjli.dylib

In [18]:
# 형태소 분석기 초기화
okt = Okt()

# 새로운 데이터프레임 초기화
new_indexdf_columns = ["Name", "index", "weights"]
new_indexdf = pd.DataFrame(columns=new_indexdf_columns)

# TF-IDF 벡터화 클래스 정의
class TfidfVectorizerWithNames(TfidfVectorizer):
    def fit_transform(self, X, y=None):
        tfidf_matrix = super().fit_transform(X, y)
        self.index_to_word = {idx: word for word, idx in self.vocabulary_.items()}
        return tfidf_matrix

    def get_feature_names_out(self, input_features=None):
        return [self.index_to_word[idx] for idx in range(len(self.index_to_word))]

# TF-IDF 벡터화 모델 초기화
vectorizer = TfidfVectorizerWithNames()

# TF-IDF 가중치 계산
tfidf_matrix = vectorizer.fit_transform(df['content'].apply(lambda x: " ".join(okt.nouns(str(x)))))

FileNotFoundError: [Errno 2] JVM DLL not found: /Library/Java/JavaVirtualMachines/adoptopenjdk-11.jdk/Contents/Home/lib/jli/libjli.dylib

In [9]:
# 결과 출력
for index, row in df.iterrows():
    content = row['content']
    tfidf_scores = np.array(tfidf_matrix[index].todense())[0]

    # TF-IDF 가중치가 높은 순으로 정렬하여 상위 20개 추출
    top_indices = tfidf_scores.argsort()[-20:][::-1]

    # 새로운 데이터프레임에 추가
    for idx in top_indices:
        row_data = [content, vectorizer.get_feature_names_out()[idx], tfidf_scores[idx]]
        new_indexdf = pd.concat([new_indexdf, pd.DataFrame([row_data], columns=new_indexdf_columns)], ignore_index=True)

# 가중치가 0.1을 넘지 않는 행 제거
new_indexdf = new_indexdf[new_indexdf['weights'] > 0.1]

# 결과 출력
print(new_indexdf)

NameError: name 'tfidf_matrix' is not defined

In [ ]:
# 'place_ids'별로 인덱스들을 모아서 저장하는 데이터프레임 초기화
indexdf = pd.DataFrame(columns=['Name', 'indices'])

# 'place_ids'별로 인덱스들을 모아서 데이터프레임에 추가
for Name, group_df in new_indexdf.groupby('Name'):
    indices = ', '.join(map(str, group_df['index'].tolist()))
    indexdf = pd.concat([indexdf, pd.DataFrame({'Name': [Name], 'indices': [indices]})], ignore_index=True)

# 'weights' 열을 제거
indexdf = indexdf.drop('weights', axis=1, errors='ignore')

# 결과 출력
print(indexdf)

In [ ]:
# Merge place_id_index_df and indexdf_source based on 'Name'
df_index = pd.merge(df, indexdf, how='left', left_on='PrimaryKey', right_on='Name')
# 'Name' 열 삭제
df_index.drop('Name_y', axis=1, inplace=True)
df_index